In [11]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
import time
import numpy as np
import mysql.connector

In [12]:
test_url='https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2012-13&SeasonType=Regular%20Season&StatCategory=PTS'

In [13]:
r = requests.get(url=test_url).json()

In [14]:
table_headers = r['resultSet']['headers']
table_values = r['resultSet']['rowSet']

In [15]:
df_cols = ['Year', 'Season_type'] + table_headers

In [16]:
headers = {
    'Accept' : '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection' : 'keep-alive',
    'Host': 'stats.nba.com',
    'Origin':'https://www.nba.com',
    'Referer': 'https://www.nba.com/',
    'Sec-Ch-Ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "Windows",
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}

In [17]:
df = pd.DataFrame(columns=df_cols)
season_types = ['Regular%Season', 'Playoffs']
years = ['2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23' ]
begin_loop = time.time()
for y in years:
    for s in season_types:
        api_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season='+y+'&SeasonType='+s+'&StatCategory=PTS'
        r = requests.get(url=api_url, headers=headers).json()
        temp_df1 = pd.DataFrame(table_values, columns=table_headers)
        temp_df2 =pd.DataFrame({'Year':[y for i in range(len(temp_df1))],
                                'Season_type' : [s for i in range(len(temp_df1))]})
        temp_df3 = pd.concat([temp_df2, temp_df1], axis=1)
        df = pd.concat([df, temp_df3], axis=0)
        print(f'Finished scraping data for the {y} {s}.')
        lag = np.random.uniform(low=5, high=20)
        print(f'...waiting {round(lag,1)} seconds')
        time.sleep(lag)
print(f'Process finished! Total run time: {round((time.time()-begin_loop)/60,2)}')

Finished scraping data for the 2012-13 Regular%Season.
...waiting 18.9 seconds
Finished scraping data for the 2012-13 Playoffs.
...waiting 17.7 seconds
Finished scraping data for the 2013-14 Regular%Season.
...waiting 7.5 seconds
Finished scraping data for the 2013-14 Playoffs.
...waiting 19.9 seconds
Finished scraping data for the 2014-15 Regular%Season.
...waiting 10.8 seconds
Finished scraping data for the 2014-15 Playoffs.
...waiting 13.6 seconds
Finished scraping data for the 2015-16 Regular%Season.
...waiting 19.1 seconds
Finished scraping data for the 2015-16 Playoffs.
...waiting 14.5 seconds
Finished scraping data for the 2016-17 Regular%Season.
...waiting 16.5 seconds
Finished scraping data for the 2016-17 Playoffs.
...waiting 5.4 seconds
Finished scraping data for the 2017-18 Regular%Season.
...waiting 19.6 seconds
Finished scraping data for the 2017-18 Playoffs.
...waiting 9.3 seconds
Finished scraping data for the 2018-19 Regular%Season.
...waiting 6.1 seconds
Finished scra

In [20]:
df.to_csv('C:/Users/tobis/Documents/nba_players_data.csv', index=False)

In [18]:
db = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='****',
    database='nb_player_stats'
)
mycursor = db.cursor()

In [29]:
df.rename(columns={'TEAM': 'TEAM_NAME'}, inplace=True)

In [30]:
for index, row in df.iterrows():
    columns = ', '.join(['`' + col + '`' for col in df.columns])
    values = tuple(row)
    placeholders = ', '.join(['%s'] * len(row))
    
    sql = f"INSERT INTO nba_players_data ({columns}) VALUES ({placeholders})"
    mycursor.execute(sql, values)


In [31]:
db.commit()

In [32]:
mycursor.close()
db.close()